In [34]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os
import cv2
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd

In [35]:
munich_imgs = os.listdir("scraped_images_grayscaled_big")
furnitures = [r for r in munich_imgs if "linz" and "cp13150" in r ]
furnitures

['1949_33905_id=cp131507_linz.jpg',
 '0711_4263_id=cp131504_linz.jpg',
 '0271_1458_id=cp131506_linz.jpg',
 '0606_3010_id=cp131502_linz.jpg',
 '0791_4970_id=cp131503_linz.jpg',
 '1950_33912_id=cp131509_linz.jpg',
 '1141_8563_id=cp131500_linz.jpg',
 '0270_1455_id=cp131505_linz.jpg',
 '1153_8691_id=cp131501_linz.jpg',
 '1950_33908_id=cp131508_linz.jpg']

In [36]:
# #vgg models (16 and 19 similar, but 16 is better)
# # model = torchvision.models.vgg16(pretrained=True)
# # model = torchvision.models.vgg19(pretrained=True)

# model.features[0] = nn.Conv2d(1,64,kernel_size=(3,3), stride=(1,1),padding=(1,1))
# model = nn.Sequential(*[*list(model.children())[:-1][0][:-10]])

# def preprocess_image(image_path):
#     """
#     This function takes a path to a single image, it then resizes it to size 50x50 \
#     and normalizes it to the range [0,1]. Lastly, it adds an extra dimension to the image \
#     which represents the batch size. These steps are needed, because we want to pass the image \
#     to a CNN. 
#     """
    
#     img = cv2.imread(image_path, -1)
#     clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
#     img = clahe.apply(img)
#     _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
#     contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)
#     mask = np.ones(img.shape, np.uint8)
#     mask.fill(255)
#     cv2.drawContours(mask, contours, 0, 0, -1)
#     img = cv2.add(thresh, mask)
#     kernel = np.ones((5,5), dtype=np.uint8)
#     img = cv2.erode(img, kernel, 10)
#     img = np.abs(np.max(img) - img)
    
#     img = cv2.resize(img, (50, 50), interpolation=cv2.INTER_AREA)
#     preprocess = torchvision.transforms.Compose([
#         torchvision.transforms.ToTensor()
#     ])
#     img = preprocess(img).unsqueeze(0)
#     return img

# def extract_features(image_path):
#     """
#     This function takes a path to a single image, it then preprocesses the image with the \
#     function preprocess_image. Afterwards it passes the image to the pretrained CNN to extract \
#     a feature descriptor. 
#     """
    
#     img = preprocess_image(image_path)
#     with torch.no_grad():
#         features = model(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     This function takes the feature descriptor and normalizes it. This is needed as we want \
#     to compute the dot-product similarity between feature descriptors of different images. \
#     And for similarity it is convenient to have all pixels on the same scale without too \
#     much magnitude differences and this also ensures stability. 
#     """
    
#     return features / np.linalg.norm(features)
    

In [37]:
# # Resnet models (gives high scores regardless)
# # model = torchvision.models.resnet50(pretrained=True)
# # model = torchvision.models.resnet101(pretrained=True) 
# # model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# # model = nn.Sequential(*list(model.children())[:-1])

# def preprocess_image(image_path):
#     """
#     This function takes a path to a single image, it then resizes it to size 50x50 \
#     and normalizes it to the range [0,1]. Lastly, it adds an extra dimension to the image \
#     which represents the batch size. These steps are needed, because we want to pass the image \
#     to a CNN. 
#     """
    
#     img = cv2.imread(image_path, -1)
#     clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
#     img = clahe.apply(img)
#     _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
#     contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)
#     mask = np.ones(img.shape, np.uint8)
#     mask.fill(255)
#     cv2.drawContours(mask, contours, 0, 0, -1)
#     img = cv2.add(thresh, mask)
#     kernel = np.ones((5,5), dtype=np.uint8)
#     img = cv2.erode(img, kernel, 10)
#     img = np.abs(np.max(img) - img)
    
#     img = cv2.resize(img, (50, 50), interpolation=cv2.INTER_AREA)
#     preprocess = torchvision.transforms.Compose([
#         torchvision.transforms.ToTensor()
#     ])
#     img = preprocess(img).unsqueeze(0)
#     return img

# def extract_features(image_path):
#     """
#     This function takes a path to a single image, it then preprocesses the image with the \
#     function preprocess_image. Afterwards it passes the image to the pretrained CNN to extract \
#     a feature descriptor. 
#     """
    
#     img = preprocess_image(image_path)
#     with torch.no_grad():
#         features = model(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     This function takes the feature descriptor and normalizes it. This is needed as we want \
#     to compute the dot-product similarity between feature descriptors of different images. \
#     And for similarity it is convenient to have all pixels on the same scale without too \
#     much magnitude differences and this also ensures stability. 
#     """
    
#     return features / np.linalg.norm(features)
    

In [38]:
# InceptionV3 model (potential)
model = torchvision.models.inception_v3(pretrained=True, aux_logits=True)

def preprocess_image(image_path):
    """
    Preprocesses a single image for InceptionV3 input.
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
    img = cv2.resize(img, (299, 299))  # Resize to 299x299 for InceptionV3
    img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
    # Since the model expects 3 channels, we stack the grayscale image to create 3 channels
    img = np.stack([img, img, img], axis=-1)
    
    img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
    img = np.transpose(img, (2, 0, 1))  # Transpose to (channels, height, width)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
    return img

def extract_features(image_path):
    """
    Extract features from a single image using InceptionV3.
    """
    img = preprocess_image(image_path)
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        features = model(img)
    return features.squeeze(0).numpy()

def normalize_features(features):
    """
    Normalize the extracted features.
    """
    return features / np.linalg.norm(features)


/home/jonathan/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jonathan/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [39]:
# # Mobile net v2 (gives to high of scores to everything)
# model = torchvision.models.mobilenet_v2(pretrained=True)

# # Modify the first convolutional layer to accept grayscale images
# model.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

# def preprocess_image(image_path):
#     """
#     Preprocesses a single grayscale image for MobileNetV2 input.
#     """
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
#     img = cv2.resize(img, (224, 224))  # Resize to 224x224 for MobileNetV2
#     img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
#     img = np.expand_dims(img, axis=0)  # Add channel dimension
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
#     return img

# def extract_features(image_path):
#     """
#     Extract features from a single image using MobileNetV2.
#     """
#     img = preprocess_image(image_path)
#     model.eval()  # Set model to evaluation mode
#     with torch.no_grad():
#         features = model.features(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     Normalize the extracted features.
#     """
#     return features / np.linalg.norm(features)

In [40]:
# # DenseNet (gives similar scores to everything)
# model = torchvision.models.densenet121(pretrained=True)

# # Modify the first convolutional layer to accept grayscale images
# model.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# def preprocess_image(image_path):
#     """
#     Preprocesses a single grayscale image for DenseNet input.
#     """
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
#     img = cv2.resize(img, (224, 224))  # Resize to 224x224 for DenseNet
#     img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
#     img = np.expand_dims(img, axis=0)  # Add channel dimension
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
#     return img

# def extract_features(image_path):
#     """
#     Extract features from a single image using DenseNet.
#     """
#     img = preprocess_image(image_path)
#     model.eval()  # Set model to evaluation mode
#     with torch.no_grad():
#         features = model.features(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     Normalize the extracted features.
#     """
#     return features / np.linalg.norm(features)


In [41]:
# # Alexnet
# model = torchvision.models.alexnet(pretrained=True)

# # Modify the first convolutional layer to accept grayscale images
# model.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

# def preprocess_image(image_path):
#     """
#     Preprocesses a single grayscale image for AlexNet input.
#     """
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
#     img = cv2.resize(img, (224, 224))  # Resize to 224x224 for AlexNet
#     img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
#     img = np.expand_dims(img, axis=0)  # Add channel dimension
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
#     return img

# def extract_features(image_path):
#     """
#     Extract features from a single image using AlexNet.
#     """
#     img = preprocess_image(image_path)
#     model.eval()  # Set model to evaluation mode
#     with torch.no_grad():
#         features = model.features(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     Normalize the extracted features.
#     """
#     return features / np.linalg.norm(features)

In [42]:
list(model.children())

[BasicConv2d(
   (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 BasicConv2d(
   (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPo

In [43]:
nk_testset = os.listdir("nk_testset")
munich_testset = os.listdir("munich_testset")

def compute_similarities_testsets(munich_testset, nk_testset, 
                                  munich_path="munich_testset", 
                                  nk_path="nk_testset"):
    """
    This function takes four arguments: 
    - munich_testset, which contains 5 grayscaled images from the munich database.
    - nk_testset, which contains 5 grayscaled images from the nk collection API.
    - munich path, the path to the directory of the munich images. 
    - nk_path, the path to the directory of the nk images. 
    
    It then computes the feature descriptors for the munich images and all the \
    nk collection images. Afterwards takes the dot-product to get the dot-product similiarity. 
    It then saves the similarity and the two images as key-value pairs in a dictionary. 
    """
    
    similarities = {}
    for nk_img in nk_testset:
        nk_img_path = os.path.join(nk_path, nk_img)
        for munich_img in munich_testset:
            munich_img_path = os.path.join(munich_path, munich_img)
            nk_img_feature_descriptor = normalize_features(extract_features(nk_img_path).flatten())
            munich_img_feature_descriptor = normalize_features(extract_features(munich_img_path).flatten())
            similarity = np.dot(
                nk_img_feature_descriptor,
                munich_img_feature_descriptor
            )
            similarities[(nk_img, munich_img)] = similarity.item()
        
    return similarities
    
sims = compute_similarities_testsets(munich_testset, nk_testset)
sims

{('speeltafel_nk.png', 'kast_mccp.jpg'): -0.06027171388268471,
 ('speeltafel_nk.png', 'dressoir_mccp.jpg'): -0.0602254681289196,
 ('speeltafel_nk.png', 'tafel_mccp.jpg'): 0.3408922553062439,
 ('speeltafel_nk.png', 'speeltafel_mccp.png'): 0.4637216031551361,
 ('speeltafel_nk.png', 'stoel_mccp.jpg'): 0.23356740176677704,
 ('dressoir_nk.jpg', 'kast_mccp.jpg'): 0.3234820067882538,
 ('dressoir_nk.jpg', 'dressoir_mccp.jpg'): 0.5264071226119995,
 ('dressoir_nk.jpg', 'tafel_mccp.jpg'): 0.26638662815093994,
 ('dressoir_nk.jpg', 'speeltafel_mccp.png'): 0.24063123762607574,
 ('dressoir_nk.jpg', 'stoel_mccp.jpg'): 0.1961786448955536,
 ('kast_nk.jpg', 'kast_mccp.jpg'): 0.7655638456344604,
 ('kast_nk.jpg', 'dressoir_mccp.jpg'): 0.3231736421585083,
 ('kast_nk.jpg', 'tafel_mccp.jpg'): 0.18063046038150787,
 ('kast_nk.jpg', 'speeltafel_mccp.png'): 0.08670418709516525,
 ('kast_nk.jpg', 'stoel_mccp.jpg'): 0.07595200091600418,
 ('tafel_nk.jpg', 'kast_mccp.jpg'): 0.2031152993440628,
 ('tafel_nk.jpg', 'dress

In [44]:
def get_table(sims):
    """
    This function takes the output produced by either the compute_similarities \ 
    or compute_similarities_testsets function, and returns a pandas dataframe/table \
    and also saves it in excel.
    """
    
    data = {}
    rows = []

    for key, value in sims.items():
        if key[0] not in data:
            data[key[0]] = []
        if key[1] not in rows:
            rows.append(key[1])
        data[key[0]].append(value)
        
    data = {key[:key.rfind(".")]:value for key, value in data.items()}
    rows = [row[:row.rfind(".")] for row in rows]
        
    df = pd.DataFrame(data, index=rows)
    #df.to_excel('output.xlsx')
    return df.T
    
get_table(sims)

,kast_mccp,dressoir_mccp,tafel_mccp,speeltafel_mccp,stoel_mccp
speeltafel_nk,-0.060272,-0.060225,0.340892,0.463722,0.233567
dressoir_nk,0.323482,0.526407,0.266387,0.240631,0.196179
kast_nk,0.765564,0.323174,0.180630,0.086704,0.075952
tafel_nk,0.203115,0.200774,0.539225,0.449782,0.365522
stoel_nk,0.179448,0.111861,0.257002,0.166814,0.667766
